# Implementing Quantum One-time pad with Cirq

This notebook is based on [lecture notes from TU Delft university](https://ocw.tudelft.nl/wp-content/uploads/LN_Week1.pdf)

Today we will see simple quantum encryption technique which is analogous to a one-time pad.

## What is _classical_ one-time pad?

First, let us understand and implement the idea behind the classical algorithm. We will consider the following scenario - Alice and Bob want to share a secret message between them. The one-time pad is a technique that will take this message, together with a _secret key_ and produce an encrypted message.

The _secret key_ is simply a binary string of the same length as the secret message. What we've done here is basically "abstracted out" the problem of encoding the message to a provider of the secret key. To ensure that the eavesdropper cannot learn the content of the message, we must have secret keys that are only known to Alice and Bob.

But - technically the idea is simple, we will take a binary string message and add (mod 2)/XOR the secret key to create encrypted message

In [1]:
import numpy as np

secret_key_known_only_to_alice_and_bob = np.array([0, 1, 1, 0, 1, 0, 1, 0])
alice_message = np.array([1, 0, 1, 0, 1, 1, 0, 1])

encrypted_message = alice_message ^ secret_key_known_only_to_alice_and_bob # XOR operation
print(encrypted_message)

[1 1 0 0 0 1 1 1]


Of course now we need a way for Bob to decrypt the message. The interesting fact about XOR operation is that if we apply it twice, we will end up with original string, so `(a ^ b ^ b == a)`. By our assumption Alice and Bob fully securely share `secret_key_known_only_to_alice_and_bob`. We hence can do the following.

In [3]:
bob_decrypted_message = encrypted_message ^ secret_key_known_only_to_alice_and_bob
print("Bob decoded message:", bob_decrypted_message, "Is Bob and Alice message equal?", np.all(alice_message == bob_decrypted_message))

Bob decoded message: [1 0 1 0 1 1 0 1] Is Bob and Alice message equal? True


## Jumping into quantum world

As we can see the algorithm is straight forward. We still need to assume that Alice and Bob shared the secret keys beforehand. First let's try naive implementation. There is no direct quantum analog of `XOR` gate, so we will use the _bit flip_ it simply changes $| 0 \rangle$ to $| 1 \rangle$ and $| 1 \rangle$ to $| 0 \rangle$.

In [5]:
import cirq
alice_qubits = cirq.NamedQubit.range(8, prefix="alice_")
